In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as T
import torch.optim as optim
import torchvision.models as models

In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F

class InceptionV4(nn.Module):
    def __init__(self):
        super(InceptionV4,self).__init__()
        # valid padding -> no padding.
        # same padding -> using padding.

        # 32x32x3 -> 16x16x
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=2,padding=0),
            nn.Ba
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding =1),
        )
        self.left1 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.right1 = nn.Conv2d(in_channels=64,out_channels=96,kernel_size=3,stride=2)

        self.left2 = nn.Sequential(
            nn.Conv2d(in_channels=160,out_channels=64,kernel_size=1),
            nn.Conv2d(in_channels=64,out_channels=96,kernel_size=3,stride=1,padding=1)
        )
        self.right2 = nn.Sequential(
            nn.Conv2d(in_channels=160,out_channels=64,kernel_size=1),
            # nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(7,1)),
            # nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(1,7)),
            nn.Conv2d(in_channels=64,out_channels=96,kernel_size=3,stride=1,padding=1),
        )
        self.left3 = nn.Conv2d(in_channels=192,out_channels=192,kernel_size=3,stride=1,padding=1)
        self.right3 = nn.MaxPool2d(kernel_size=3,stride=1,padding=1)

        self.inc_A = Inception_A(384)
        self.red_A = Reduction_A(384)
        self.inc_B = Inception_B(1024)
        self.red_B = Reduction_B(1024)
        self.inc_C = Inception_C(1536)

        self.fc = nn.Linear(1536,10)
        self.classifier = nn.Sequential(
            nn.Linear(1536,512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Linear(512,100)
        )
        self.sm = nn.Softmax(dim=1)
    def forward(self,x):
        x = self.stem(x)
        left1 = self.left1(x)
        right1 = self.right1(x)

        concat1 = [left1,right1]
        concat1 = torch.cat(concat1,1)
        #Filter concat
        left2 = self.left2(concat1)
        right2 = self.right2(concat1)
        concat2 = [left2,right2]
        concat2 = torch.cat(concat2,1)
        #Filter concat
        left3 = self.left3(concat2)
        right3 = self.right3(concat2)
        concat3 = [left3,right3]
        concat3 = torch.cat(concat3,1)
        #Filter concat
        #### stem Finish###
        a = concat3
        for i in range(4):
            a = self.inc_A(concat3)
        a = self.red_A(a)
        for i in range(7):
            a = self.inc_B(a)
        a = self.red_B(a)
        for i in range(3):
            a = self.inc_C(a)
        a = F.adaptive_avg_pool2d(a,(1,1))
        a = a.view(a.size(0),-1)
        a = self.classifier(a)
        # softmax를 사용하지 않으면
        # adaptive avg pool이 무슨 역할ㅇ르 하는지
        
        return a

class Inception_A(nn.Module):
    def __init__(self,in_channels):
        super(Inception_A,self).__init__()
        self.layer1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels=in_channels,out_channels=96,kernel_size=1,stride=1,padding=0)
        )
        self.layer2 = nn.Conv2d(in_channels=in_channels,out_channels=96,kernel_size=1,stride=1)
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=64,kernel_size=1),
            nn.Conv2d(in_channels=64,out_channels=96,kernel_size=3,stride=1,padding=1),
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=64,kernel_size=1),
            nn.Conv2d(in_channels=64,out_channels=96,kernel_size=3,padding=1),
            nn.Conv2d(in_channels=96,out_channels=96,kernel_size=3,padding=1),
        )
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        layer3 = self.layer3(x)
        layer4 = self.layer4(x)
        concat = [layer1,layer2,layer3,layer4]
        return torch.cat(concat,1)
# 7x1 대신 3x1 로 변경할 예정
class Inception_B(nn.Module):
    def __init__(self,in_channels):
        super(Inception_B,self).__init__()
        self.layer1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels=in_channels,out_channels=128,kernel_size=1),
        )
        self.layer2 = nn.Conv2d(in_channels=in_channels,out_channels=384,kernel_size=1)
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=192,kernel_size=1),
            nn.Conv2d(in_channels=192,out_channels=224,kernel_size=(1,3),padding=(0,1)),
            nn.Conv2d(in_channels=224,out_channels=256,kernel_size=(3,1),padding=(1,0)),
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=192,kernel_size=1),
            nn.Conv2d(in_channels=192,out_channels=192,kernel_size=(1,3),padding=(0,1)),
            nn.Conv2d(in_channels=192,out_channels=224,kernel_size=(3,1),padding=(1,0)),
            nn.Conv2d(in_channels=224,out_channels=224,kernel_size=(1,3),padding=(0,1)),
            nn.Conv2d(in_channels=224,out_channels=256,kernel_size=(3,1),padding=(1,0)),
        )
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        layer3 = self.layer3(x)
        layer4 = self.layer4(x)
        concat = [layer1,layer2,layer3,layer4]
        return torch.cat(concat,1)

class Inception_C(nn.Module):
    def __init__(self,in_channels):
        super(Inception_C,self).__init__()
        self.layer1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels=in_channels,out_channels=256,kernel_size=1),
        )
        self.layer2= nn.Conv2d(in_channels=in_channels,out_channels=256,kernel_size=1)
        self.layer3_1 = nn.Conv2d(in_channels=in_channels,out_channels=384,kernel_size=1)
        self.layer3_2_left = nn.Conv2d(in_channels=384,out_channels=256,kernel_size=(1,3),padding=(0,1))
        self.layer3_2_right = nn.Conv2d(in_channels=384,out_channels=256,kernel_size=(3,1),padding=(1,0))
        self.layer4_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=384,kernel_size=1),
            nn.Conv2d(in_channels=384,out_channels=448,kernel_size=(1,3),padding=(0,1)),
            nn.Conv2d(in_channels=448,out_channels=512,kernel_size=(3,1),padding=(1,0)),
        )
        self.layer4_2_left = nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(3,1),padding=(1,0))
        self.layer4_2_right = nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,3),padding=(0,1))

    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        layer3 = self.layer3_1(x)
        layer3_l = self.layer3_2_left(layer3)
        layer3_r = self.layer3_2_right(layer3)
        layer4 = self.layer4_1(x)
        layer4_l=self.layer4_2_left(layer4)
        layer4_r = self.layer4_2_right(layer4)

        concat = [layer1,layer2,layer3_l,layer3_r,layer4_l,layer4_r]
        return torch.cat(concat,1)
# 11 -> 4
class Reduction_A(nn.Module):
    def __init__(self,in_channels):
        super(Reduction_A,self).__init__()
        self.layer1 = nn.MaxPool2d(kernel_size=3,stride=2,padding =0)
        self.layer2 = nn.Conv2d(in_channels=in_channels,out_channels=384,kernel_size=3,stride=2)
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=192,kernel_size=1),
            nn.Conv2d(in_channels=192,out_channels=224,kernel_size=3,padding=1),
            nn.Conv2d(in_channels=224,out_channels=256,kernel_size=3,stride=2,padding=0)
        )
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        layer3 = self.layer3(x)
        concat = [layer1,layer2,layer3]
        concat = torch.cat(concat,1)
        return concat

# 4->2
class Reduction_B(nn.Module):
    def __init__(self,in_channels):
        super(Reduction_B,self).__init__()
        self.layer1 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=192,kernel_size=1),
            nn.Conv2d(in_channels=192,out_channels=192,kernel_size=3,stride=2,padding=1),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=256,kernel_size=1),
            nn.Conv2d(in_channels=256,out_channels=320,kernel_size=3,stride=2,padding=1)
        )
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        layer3 = self.layer3(x)
        
        concat = [layer1,layer2,layer3]
        concat = torch.cat(concat,1)
        return concat


In [ ]:
transforms = T.Compose([
                        T.RandomCrop((32,32),padding=(4,4)),
                        T.RandomHorizontalFlip(p=0.5),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                        
])

trainset = dsets.CIFAR100(root = './datasets',train=True,transform=transforms,download=True)
testset = dsets.CIFAR100(root='./datasets',train=False,transform=T.Compose([         #T.RandomCrop((32,32),padding=(4,4)),
                                                                                                                    # T.RandomHorizontalFlip(p=0.5),
                                                                                                                    T.ToTensor(),
                                                                                                                    T.Normalize(mean=[0.485,0.456,0.406],
                                                                                                                                std = [0.229,0.224,0.225])
]),download=True)
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainLoader = torch.utils.data.DataLoader(trainset,batch_size = 128,shuffle = True)
testLoader = torch.utils.data.DataLoader(testset,batch_size =128,shuffle=False,drop_last=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = InceptionV4()
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr=0.05,momentum=0.9,weight_decay=5e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

In [ ]:

def check_accuracy(network,testLoader):
    class_correct = list(0. for i in range(100))
    class_total = list(0. for i in range(100))
    with torch.no_grad():
        for data in testLoader:
            images,labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = network(images)
            print(outputs.shape)
            print(outputs)
        
            #
            _,predicted = torch.max(outputs,1)
            print(outputs[predicted])
            c = (predicted == labels).squeeze()
            for i in range(128):
                label = labels[i]
                class_correct[label] +=c[i].item()
                class_total[label] +=1
            
    accuracy_sum = 0
    for i in range(10):
        temp = 100* class_correct[i] / class_total[i]
        accuracy_sum += temp
    return accuracy_sum / 10

In [ ]:

def show_accuracy(network,testLoader):
    
    class_correct = list(0. for i in range(100))
    class_total = list(0. for i in range(100))
    with torch.no_grad():
        for data in testLoader:
            images,labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = network(images)
            #
            print(outputs)
            a,predicted = torch.max(outputs,1)
            print(a)
            print(a.shape)
            print(predicted)
            print(predicted.shape)
            c = (predicted == labels).squeeze()
            for i in range(128):
                label = labels[i]
                class_correct[label] +=c[i].item()
                class_total[label] +=1
    accuracy_sum = 0
    for i in range(10):
        temp = 100* class_correct[i] / class_total[i]
        print('Accuracy of %5s : %2d %%'%(classes[i],temp))
        accuracy_sum += temp
    print('Accuracy average: ',accuracy_sum / 10)

In [ ]:
epochs_accuracy_list = []
epochs_accuracy = 0
lr = 0.05
for epoch in range(200):
    running_loss = 0.0
    avg_loss=0.0
    for i,data in enumerate(trainLoader,0):
        #get the inputs
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        #zero the parameter gradients
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # # print statistics
        # running_loss += loss.item()
        # avg_loss += loss.item()
        # if i%50 ==49:
        #     print('[%d,%5d] loss: %.3f'%(epoch+1,i+1,running_loss/50))
        #     running_loss=0.0

    # if abs(pre_avg_loss-avg_loss)<=0.01 and epoch!=0:
    #     print("learning rate 50% reduced.")
    #     scheduler.step()
    acc = check_accuracy(net,testLoader)
    print("epoch: {} | Accuracy: {}".format(epoch,acc))
    epochs_accuracy_list.append(acc)
    
    if epoch % 5 == 0:
        show_accuracy(net,testLoader)
print('Finished Training')



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [ 0.0161, -0.0081, -0.0270,  ..., -0.0159,  0.0081,  0.0045],
        [ 0.0166, -0.0115, -0.0250,  ..., -0.0162,  0.0040,  0.0035],
        ...,
        [ 0.0108, -0.0084, -0.0224,  ..., -0.0142,  0.0065,  0.0028],
        [ 0.0171, -0.0083, -0.0269,  ..., -0.0149,  0.0057,  0.0037],
        [ 0.0178, -0.0095, -0.0246,  ..., -0.0153,  0.0058,  0.0040]],
       device='cuda:0')
tensor([[ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051],
        [ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051],
        [ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051],
        ...,
        [ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051],
        [ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051],
        [ 0.0155, -0.0104, -0.0245,  ..., -0.0144,  0.0038,  0.0051]],
       device='cuda:0')
torch.Size([128, 10])
tensor([[ 0.0125, -0.0094, -0.0238,  ..., -0.0139,  0.0056,  0.0018],
        [ 0.0190, -0.0111,

KeyboardInterrupt: ignored

In [ ]:
show_accuracy(net,testLoader)